<a href="https://colab.research.google.com/github/thislis/textdenoise/blob/main/%ED%83%90%EB%85%BC_%EC%BD%94%EB%93%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

링크 : https://wikidocs.net/24996

In [1]:
pip install git+https://github.com/ssut/py-hanspell.git

  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-myt5pvuu
  Running command git clone -q https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-myt5pvuu


In [2]:
import pandas as pd
import shutil
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from hanspell import spell_checker

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
lines = pd.read_csv('/content/drive/MyDrive/단어만 8만개.csv', names=['A', 'B'], sep=',')
print(lines)

           A      B
0       가와가현   가가와현
1       가대가소   가가대소
2       가문가전   가가문전
3      가붓자가식  가가붓자식
4       가성가자   가가성자
...      ...    ...
81390   힘이받천   힘받이천
81391   힘이받침   힘받이침
81392   힘잇받감   힘받잇감
81393   힝항뚱뚱   힝뚱항뚱
81394   힝힝뚱뚱   힝뚱힝뚱

[81395 rows x 2 columns]


In [5]:
lines = lines.loc[:, 'A':'B']
lines = lines[0:80000]
lines.sample(10)

,A,B
39775,삼대수엽,삼수대엽
20867,들나충무,들충나무
76878,필잡원기,필원잡기
72964,키코스니파,키니코스파
59822,일양거전,일거양전
60670,입양신명,입신양명
6983,관놀자이,관자놀이
25609,머기리사,머리기사
37177,빙교상예,빙상교예
67619,진팔복단,진복팔단


In [6]:
lines.B = lines.B.apply(lambda x : '\t '+ str(x) + ' \n')
lines.sample(10)

,A,B
288,가이랑표,\t 가랑이표 \n
46554,실부제피,\t 실제부피 \n
54074,오방놀진이,\t 오방진놀이 \n
35382,불고갈리,\t 불갈고리 \n
33254,변무화궁,\t 변화무궁 \n
10358,기전만술,\t 기만전술 \n
17560,대자용재,\t 대용자재 \n
33297,별우달배편,\t 별배달우편 \n
34203,볼볼뚝뚝,\t 볼뚝볼뚝 \n
36337,비딱공지,\t 비공딱지 \n


In [7]:
# 글자 집합 구축
src_vocab=set()
for l in lines.A: # 1줄씩 읽음
  for char in str(l): # 1개의 글자씩 읽음
    src_vocab.add(char)

tar_vocab=set()
for l in lines.B:
  for char in l:
     tar_vocab.add(char)

print(src_vocab, tar_vocab)

{'촛', '쌘', '몬', '명', '잣', '쉑', '깝', '웽', '러', '층', '님', '섯', '쏘', '쯔', '엑', '길', '퀸', '랍', '높', '젹', '퓰', '쇽', '구', '곶', '탁', '벚', '텟', '랩', '있', '황', '루', '책', '졸', '릉', '군', '쿼', '헙', '반', '뇽', '랒', '징', '쓸', '십', '돗', '땃', '헹', '닐', '펴', '성', '죤', '서', '컹', '매', '얄', '훈', '켸', '농', '뎝', '졧', '촘', '볶', '통', '짇', '앨', '튿', '펭', '붉', '씀', '뺄', '담', '기', '만', '맑', '괄', '바', '쌈', '쭉', '낌', '멘', '뽐', '덥', '뼛', '휴', '엄', '짼', '지', '찰', '돤', '첫', '넷', '뢰', '렌', '옌', '릿', '퀴', '챈', '겁', '땀', '행', '컨', '팀', '뇨', '음', '텃', '향', '앵', '턱', '브', '아', '휠', '맛', '웃', '칠', '늬', '젯', '얌', '욱', '주', '니', '큐', '껭', '배', '탕', '풀', '어', '튕', '견', '셕', '막', '앳', '뇰', '샐', '뒤', '뭏', '뎬', '활', '엠', '적', '릭', '컴', '듀', '쌔', '젝', '앞', '샤', '놋', '뎅', '렝', '답', '늘', '척', '찐', '엣', '꾀', '쾰', '팁', '동', '캔', '취', '걀', '랠', '촌', '뻐', '겉', '짬', '곳', '딴', '태', '톡', '불', '펙', '뚜', '작', '겡', '널', '윌', '샅', '훅', '앚', '퍼', '퀘', '펠', '뭄', '율', '닫', '멤', '질', '헛', '꿩', '뫼', '를', '폐', '깍', '섹', '빠', '펫', '툭', '스', '분', '치',

In [8]:
src_vocab_size = len(src_vocab)+1
tar_vocab_size = len(tar_vocab)+1
print(src_vocab_size)
print(tar_vocab_size)

1684
1687


In [9]:
src_vocab = sorted(list(src_vocab))
tar_vocab = sorted(list(tar_vocab))
print(src_vocab)
print(tar_vocab)

['ㅅ', '가', '각', '간', '갈', '갉', '감', '갑', '값', '갓', '강', '갖', '같', '개', '객', '갠', '갤', '갬', '갭', '갯', '갱', '갸', '걀', '걍', '거', '걱', '건', '걷', '걸', '검', '겁', '것', '겅', '겆', '겉', '게', '겐', '겔', '겝', '겟', '겡', '겨', '격', '겪', '견', '결', '겸', '겹', '겻', '경', '곁', '계', '곗', '고', '곡', '곤', '곧', '골', '곪', '곬', '곰', '곱', '곳', '공', '곶', '과', '곽', '관', '괄', '광', '괘', '괭', '괴', '괵', '굉', '교', '구', '국', '군', '굳', '굴', '굵', '굼', '굽', '굿', '궁', '궂', '궈', '권', '궐', '궤', '귀', '귄', '귐', '귓', '규', '균', '귤', '그', '극', '근', '귿', '글', '긁', '긃', '금', '급', '긋', '긍', '긔', '긜', '긧', '기', '긴', '길', '김', '깁', '깃', '깅', '깆', '깊', '까', '깍', '깎', '깐', '깔', '깜', '깝', '깟', '깡', '깥', '깨', '깩', '깬', '깰', '깸', '깻', '깽', '꺄', '꺌', '꺼', '꺽', '꺾', '껀', '껄', '껌', '껍', '껏', '껑', '께', '껭', '껴', '껸', '꼐', '꼬', '꼭', '꼰', '꼴', '꼼', '꼽', '꼿', '꽁', '꽂', '꽃', '꽈', '꽐', '꽝', '꽤', '꽹', '꾀', '꾐', '꾸', '꾹', '꾼', '꿀', '꿇', '꿈', '꿉', '꿍', '꿔', '꿜', '꿩', '꿰', '뀀', '뀌', '뀐', '뀜', '끄', '끈', '끊', '끌', '끎', '끓', '끔', '끗', '끝', '끼', '끽', '낄', '낌',

In [10]:
src_to_index = dict([(word, i+1) for i, word in enumerate(src_vocab)])
tar_to_index = dict([(word, i+1) for i, word in enumerate(tar_vocab)])
print(src_to_index)
print(tar_to_index)

{'ㅅ': 1, '가': 2, '각': 3, '간': 4, '갈': 5, '갉': 6, '감': 7, '갑': 8, '값': 9, '갓': 10, '강': 11, '갖': 12, '같': 13, '개': 14, '객': 15, '갠': 16, '갤': 17, '갬': 18, '갭': 19, '갯': 20, '갱': 21, '갸': 22, '걀': 23, '걍': 24, '거': 25, '걱': 26, '건': 27, '걷': 28, '걸': 29, '검': 30, '겁': 31, '것': 32, '겅': 33, '겆': 34, '겉': 35, '게': 36, '겐': 37, '겔': 38, '겝': 39, '겟': 40, '겡': 41, '겨': 42, '격': 43, '겪': 44, '견': 45, '결': 46, '겸': 47, '겹': 48, '겻': 49, '경': 50, '곁': 51, '계': 52, '곗': 53, '고': 54, '곡': 55, '곤': 56, '곧': 57, '골': 58, '곪': 59, '곬': 60, '곰': 61, '곱': 62, '곳': 63, '공': 64, '곶': 65, '과': 66, '곽': 67, '관': 68, '괄': 69, '광': 70, '괘': 71, '괭': 72, '괴': 73, '괵': 74, '굉': 75, '교': 76, '구': 77, '국': 78, '군': 79, '굳': 80, '굴': 81, '굵': 82, '굼': 83, '굽': 84, '굿': 85, '궁': 86, '궂': 87, '궈': 88, '권': 89, '궐': 90, '궤': 91, '귀': 92, '귄': 93, '귐': 94, '귓': 95, '규': 96, '균': 97, '귤': 98, '그': 99, '극': 100, '근': 101, '귿': 102, '글': 103, '긁': 104, '긃': 105, '금': 106, '급': 107, '긋': 108, '긍': 109, '긔': 110, '긜': 11

In [11]:
encoder_input = []
for line in lines.A: #입력 데이터에서 1줄씩 문장을 읽음
    temp_X = []
    for w in str(line): #각 줄에서 1개씩 글자를 읽음
      temp_X.append(src_to_index[w]) # 글자를 해당되는 정수로 변환
    encoder_input.append(temp_X)
print(encoder_input[:5])

[[2, 1053, 2, 1624], [2, 314, 2, 846], [2, 621, 2, 1152], [2, 720, 1128, 2, 903], [2, 828, 2, 1128]]


In [12]:
decoder_input = []
for line in lines.B:
    temp_X = []
    for w in line:
      temp_X.append(tar_to_index[w])
    decoder_input.append(temp_X)
print(decoder_input[:5])

[[1, 3, 5, 5, 1056, 1627, 3, 2], [1, 3, 5, 5, 317, 849, 3, 2], [1, 3, 5, 5, 624, 1155, 3, 2], [1, 3, 5, 5, 723, 1131, 906, 3, 2], [1, 3, 5, 5, 831, 1131, 3, 2]]


In [13]:
decoder_target = []
for line in lines.B:
    t=0
    temp_X = []
    for w in line:
      if t>0:
        temp_X.append(tar_to_index[w])
      t=t+1
    decoder_target.append(temp_X)
print(decoder_target[:5])

[[3, 5, 5, 1056, 1627, 3, 2], [3, 5, 5, 317, 849, 3, 2], [3, 5, 5, 624, 1155, 3, 2], [3, 5, 5, 723, 1131, 906, 3, 2], [3, 5, 5, 831, 1131, 3, 2]]


In [14]:
#max_src_len = max([len(line) for line in lines.A])
#max_tar_len = max([len(line) for line in lines.B])
#print(max_src_len)
#print(max_tar_len)

lenlineA = []
lenlineB = []

for line in lines.A:
  lenlineA.append(len(str(line)))

for line in lines.B:
  lenlineB.append(len(str(line)))

max_src_len = max(lenlineA)
max_tar_len = max(lenlineB)

print(lenlineA.index(max_src_len))
print(lenlineA, max_src_len)
print(lenlineB, max_tar_len)

3
[4, 4, 4, 5, 4, 4, 4, 4, 4, 5, 4, 4, 4, 4, 4, 4, 4, 5, 4, 4, 5, 4, 4, 4, 4, 5, 4, 4, 4, 5, 4, 5, 4, 4, 4, 5, 4, 4, 4, 4, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 4, 5, 4, 4, 5, 5, 4, 4, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 4, 5, 4, 4, 4, 4, 5, 5, 4, 4, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 4, 4, 5, 4, 4, 4, 5, 4, 4, 5, 5, 4, 5, 5, 5, 4, 5, 5, 4, 5, 5, 4, 5, 4, 4, 4, 5, 4, 4, 4, 4, 4, 4, 4, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 4, 5, 4, 4, 4, 4, 4, 4, 4, 4, 5, 4, 4, 4, 4, 4, 5, 4, 5, 4, 4, 4, 4, 4, 4, 4, 5, 5, 4, 4, 4, 5, 4, 5, 5, 4, 4, 4, 4, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 4, 4, 4, 5, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 4, 5, 5, 5, 4, 4, 5, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 4, 4, 4, 4, 4, 5, 5, 4, 4, 5, 4, 4, 5, 4, 4, 4, 4, 4, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 4, 4, 4, 4, 5, 4, 4, 4, 5, 4

In [15]:
encoder_input = pad_sequences(encoder_input, maxlen=max_src_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen=max_tar_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen=max_tar_len, padding='post')

In [16]:
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)

In [17]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model
import numpy as np

In [18]:
encoder_inputs = Input(shape=(None, src_vocab_size))
encoder_lstm = LSTM(units=256, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
# encoder_outputs도 같이 리턴받기는 했지만 여기서는 필요없으므로 이 값은 버림.
encoder_states = [state_h, state_c]
# LSTM은 바닐라 RNN과는 달리 상태가 두 개. 바로 은닉 상태와 셀 상태.

In [19]:
decoder_inputs = Input(shape=(None, tar_vocab_size))
decoder_lstm = LSTM(units=256, return_sequences=True, return_state=True)
decoder_outputs, _, _= decoder_lstm(decoder_inputs, initial_state=encoder_states)
# 디코더의 첫 상태를 인코더의 은닉 상태, 셀 상태로 합니다.
decoder_softmax_layer = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")

In [20]:
model.fit(x=[encoder_input, decoder_input], y=decoder_target, batch_size=128, epochs=50, validation_split=0.2)

Epoch 1/50
500/500 [==============================] - 14s 20ms/step - loss: 2.9757 - val_loss: 3.0217
Epoch 2/50
500/500 [==============================] - 7s 14ms/step - loss: 2.1805 - val_loss: 2.2040
Epoch 3/50
500/500 [==============================] - 7s 14ms/step - loss: 1.3173 - val_loss: 1.5646
Epoch 4/50
500/500 [==============================] - 7s 14ms/step - loss: 0.7541 - val_loss: 1.1362
Epoch 5/50
500/500 [==============================] - 7s 15ms/step - loss: 0.4768 - val_loss: 0.9387
Epoch 6/50
500/500 [==============================] - 7s 14ms/step - loss: 0.3258 - val_loss: 0.7788
Epoch 7/50
500/500 [==============================] - 7s 14ms/step - loss: 0.2362 - val_loss: 0.6727
Epoch 8/50
500/500 [==============================] - 7s 14ms/step - loss: 0.1789 - val_loss: 0.6163
Epoch 9/50
500/500 [==============================] - 7s 14ms/step - loss: 0.1425 - val_loss: 0.5674
Epoch 10/50
500/500 [==============================] - 7s 14ms/step - loss: 0.1161 - val_l

In [21]:
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)

In [22]:
encoder_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, 1684)]      0         
                                                                 
 lstm (LSTM)                 [(None, 256),             1987584   
                              (None, 256),                       
                              (None, 256)]                       
                                                                 
Total params: 1,987,584
Trainable params: 1,987,584
Non-trainable params: 0
_________________________________________________________________


In [23]:
# 이전 시점의 상태들을 저장하는 텐서
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
# 문장의 다음 단어를 예측하기 위해서 초기 상태(initial_state)를 이전 시점의 상태로 사용. 이는 뒤의 함수 decode_sequence()에 구현
decoder_states = [state_h, state_c]
# 훈련 과정에서와 달리 LSTM의 리턴하는 은닉 상태와 셀 상태인 state_h와 state_c를 버리지 않음.
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)

In [24]:
index_to_src = dict((i, char) for char, i in src_to_index.items())
index_to_tar = dict((i, char) for char, i in tar_to_index.items())

In [25]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # <SOS>에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1, 1, tar_vocab_size))
    target_seq[0, 0, tar_to_index['\t']] = 1

    stop_condition = False
    decoded_sentence = ""

    # stop_condition이 True가 될 때까지 루프 반복
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        # output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        
        datalist = decoder_model.predict([target_seq] + states_value)
        output_tokens = datalist[0]
        h = datalist[1]
        c = datalist[2]

        # 예측 결과를 문자로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_to_tar[sampled_token_index + 1]

        # 현재 시점의 예측 문자를 예측 문장에 추가
        decoded_sentence += sampled_char

        # <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == '\n' or len(decoded_sentence) > max_tar_len):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1, 1, tar_vocab_size))
        target_seq[0, 0, sampled_token_index] = 1

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

In [36]:
for seq_index in [1,2,3,10]: # 입력 문장의 인덱스
    input_seq = encoder_input[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    decoded_sentence = decoded_sentence.replace("ㅅ", "")
    decoded_sentence = decoded_sentence.replace("\t", "")
    print(35 * "-")
    print('입력 문장:', lines.A[seq_index])
    print('정답 문장:', lines.B[seq_index][1:len(lines.B[seq_index])-1]) # '\t'와 '\n'을 빼고 출력
    print('번역기가 번역한 문장:', decoded_sentence[:len(decoded_sentence)-1]) # '\n'을 빼고 출력

-----------------------------------
입력 문장: 가대가소
정답 문장:  가가대소 
번역기가 번역한 문장: 각각댁속
-----------------------------------
입력 문장: 가문가전
정답 문장:  가가문전 
번역기가 번역한 문장: 각각묻절
-----------------------------------
입력 문장: 가붓자가식
정답 문장:  가가붓자식 
번역기가 번역한 문장: 각각붕작신
-----------------------------------
입력 문장: 가역감관
정답 문장:  가감역관 
번역기가 번역한 문장: 각갑엮괄
